## Histone deacetylase 1 - part 2 (MMP identification)

### Import libraries

In [1]:
import pandas as pd

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import sys
sys.path.append('/home/daniel/wizepair2')
from classes.mmp import MMP

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read in the dataset

In [3]:
df = pd.read_csv('hdac1_inhibitors_stripped.csv')
len(df.index)

108

### Create cartesian product of unique molecules tested in the same assay

In [4]:
df_pairs = pd.merge(df, df, on='assay_chembl_id')
df_pairs = df_pairs[['stripped_smiles_x', 'stripped_smiles_y']].drop_duplicates()
df_pairs

,stripped_smiles_x,stripped_smiles_y
0,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...
1,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,N#Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
2,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...
3,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
4,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C)...
...,...,...
116,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(Cl...
117,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...
118,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...
119,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,Nc1ccccc1NC(=O)c1ccc(-c2ncc(CN3CCC3)cc2F)cc1


### Identify all pairs

In [5]:
df_pairs = df_pairs.sample(frac=1).parallel_apply(lambda x: MMP(x.stripped_smiles_x, x.stripped_smiles_y, strictness=6).execute(), axis=1)

In [6]:
df_pairs = pd.json_normalize(df_pairs.explode())
df_pairs.sample(3).transpose()

,415,18,144
smiles1,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C#...,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
smiles2,N#Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C#...
percentmcs,0.848485,0.787879,0.787879
radius,4.0,2.0,1.0
valid,True,False,False
solversecs,1.95574,1.376076,1.470282
embedding,"[0, 0, 0, 0, 0, 39, 222, 88, 84, 98, 14, 0, 0,...","[0, 0, 0, 0, 9, 68, 205, 59, 90, 85, 1, 0, 0, ...","[0, 0, 0, 0, 2, 58, 216, 66, 84, 93, 5, 0, 0, ..."
predsolversecs,3.698,3.112,3.31
error,None,not one2one reaction,not one2one reaction
smirks,[H]-[#6:1](-[H])(-[#6:2])-[#7:3]1-[#6:4](-[H])...,[H]-[#6](-[H])(-[H])-[#6](-[H])(-[#7]1-[#6](-[...,[H]-[#6:4](-[H])(-[H])-[#6:3].[H]-[#6:1](-[H])...


### Drop failures and write output to file

In [7]:
df_pairs = df_pairs[df_pairs.valid]
df_pairs.to_csv('hdac1_inhibitors_pairs.csv', index=False)